In [1]:
import ee
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import folium
import geemap

In [2]:
ee.Authenticate()
# Earth Engine API 초기화
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=wqw4___LNBJSyMNQjq_gzSOWA07HdGR8rfbaavHhaKw&tc=xJMTeBgocqSUspRamELG53RF7gGiPjEZMImTatymyGA&cc=Pt9axjat7CaGlQBzZMyZT_vyqHfoS2CiMajDNU9Qx4A

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkB5SRCjkDPcWEmqKgy5g04phx_kjBMVVksHlwBK7Vr7bA3XJIeBQE

Successfully saved authorization token.


In [3]:
import ee
ee.Initialize()

# Define the area of interest
coordinates = ee.Geometry.Polygon(
    [[[128.695307, 36.547018],
      [128.695307, 36.560532],
      [128.737621, 36.560532],
      [128.737621, 36.547018]]]
)

# Filter the Sentinel-1 ImageCollection for the given dates and parameters
sar_image_june = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filterBounds(coordinates)\
    .filterDate('2023-06-12', '2023-06-13')\
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
    .first()\
    .select('VV')

sar_image_july = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filterBounds(coordinates)\
    .filterDate('2023-07-18', '2023-07-19')\
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
    .first()\
    .select('VV')

# Combine the images
false_color = ee.Image.rgb(sar_image_july, sar_image_june, ee.Image.constant(0))

# Add the false color image to the map
Map = ee.Map()
Map.centerObject(coordinates, 13)
Map.addLayer(false_color, {}, 'False Color Composite')
Map.addLayer(ee.Image('Google/VWorldHybrid'), {}, 'VWorld Hybrid')
Map.addLayer(ee.Image('Google/VWorldSatellite'), {}, 'VWorld Satellite')
Map


AttributeError: ignored

In [6]:
import ee
import folium

# Google Earth Engine 초기화
ee.Initialize()

# 영역 정의
coordinates = [
    [128.695307, 36.547018],
    [128.695307, 36.560532],
    [128.737621, 36.560532],
    [128.737621, 36.547018]
]
aoi = ee.Geometry.Polygon(coordinates)

# Sentinel-1 이미지 가져오기
def get_s1_image(sdate,edate):
    return ee.ImageCollection('COPERNICUS/S1_GRD')\
        .filterBounds(aoi)\
        .filterDate(sdate,edate)\
        .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
        .first()\
        .select('VV')

# 두 날짜에 대한 이미지
s1_june = get_s1_image('2023-06-12', '2023-06-30')
s1_july = get_s1_image('2023-07-18', '2023-07-30')

# folium을 사용하여 지도에 표시
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Folium에 이 메소드를 추가
folium.Map.add_ee_layer = add_ee_layer

# 지도 생성
my_map = folium.Map(location=[36.553775, 128.716627], zoom_start=12)

# 각 날짜에 대한 이미지 레이어 추가
vis_params = {'min': -20, 'max': 0}
my_map.add_ee_layer(s1_june, vis_params, 'S1 June 12, 2023')
my_map.add_ee_layer(s1_july, vis_params, 'S1 July 18, 2023')

# 레이어 컨트롤 추가
folium.LayerControl().add_to(my_map)

# 지도 표시
my_map


In [27]:
import ee
import folium

# Google Earth Engine 초기화
ee.Initialize()

# 영역 정의
coordinates = [
    [128.695307, 36.547018],
    [128.695307, 36.560532],
    [128.737621, 36.560532],
    [128.737621, 36.547018]
]
aoi = ee.Geometry.Polygon(coordinates)

# Sentinel-1 이미지 가져오기
def get_s1_image(sdate,edate):
    return ee.ImageCollection('COPERNICUS/S1_GRD')\
        .filterBounds(aoi)\
        .filterDate(sdate,edate)\
        .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))\
        .first()\
        .select('VV')

# 두 날짜에 대한 이미지
s1_june = get_s1_image('2023-06-12','2023-06-30')
s1_july = get_s1_image('2023-07-18','2023-07-30')

# False color composite 만들기
false_color = ee.Image.rgb(s1_july, s1_june, ee.Image.constant(-20))

# Folium 지도에 추가하는 함수
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Folium에 이 메소드를 추가
folium.Map.add_ee_layer = add_ee_layer

# 지도 생성
my_map = folium.Map(location=[36.553775, 128.716627], zoom_start=12)

# False color 이미지 레이어 추가
vis_params = {'min': -20, 'max': 0}
my_map.add_ee_layer(false_color, vis_params, 'False Color Composite')

# Vworld 위성 이미지를 타일 레이어로 추가 (Vworld 타일 서버 URL 필요)
# 예시: my_map.add_tile_layer(tiles="Vworld Tile Server URL", attr='Vworld Satellite', name='Vworld Satellite')

# 레이어 컨트롤 추가
folium.LayerControl().add_to(my_map)

# 지도 표시
my_map


In [22]:
def get_min_max(image):
    # Reduce the region with increased scale and maxPixels
    stats = image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        scale=100,  # Increase the scale to reduce the number of pixels
        maxPixels=1e9  # Increase the maxPixels limit
    )
    return stats.getInfo()

# Get min and max for both images
min_max_june = get_min_max(s1_june)
min_max_july = get_min_max(s1_july)

min_max_june, min_max_july

({'VV_max': 25.518982674150593, 'VV_min': -63.131031458402596},
 {'VV_max': 27.105415835873025, 'VV_min': -36.26527355100858})

In [17]:
import ee
import geemap

# Earth Engine 초기화
ee.Initialize()

# 좌표 설정
coordinates = [
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936],
    [128.913574, 35.024936]
]

# 지정된 좌표로 지역을 생성
region = ee.Geometry.Polygon([coordinates])

# Dynamic World 데이터셋 불러오기
dynamicWorld = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")

# 2019년 1월 데이터 필터링
filteredImage = dynamicWorld.filterDate('2022-12-15', '2022-12-31').filterBounds(region)

# 필터링된 이미지의 첫 번째 이미지 가져오기
firstImage = filteredImage.first()

# 지도 생성 및 설정
Map = geemap.Map()
Map.centerObject(region, 10)
Map.addLayer(firstImage, {}, 'Dynamic World Image')

# 지도 출력
Map


Map(center=[35.05795764611553, 128.9470480000001], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
aoi = ee.Geometry.Polygon(
    [[[126.19653266055587, 36.853168924185226],
      [126.19653266055587, 36.83836240145186],
      [126.21259510596076, 36.83836240145186],
      [126.21259510596076, 36.853168924185226],
      [126.19653266055587, 36.853168924185226]]]
)